Copyright 2024 Google LLC. Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License

In [ ]:
import ee

ee.Authenticate()

In [ ]:
#@markdown Specify Earth Engine project id.
PROJECT_ID = ''  # @param {type:"string"}
ee.Initialize(project=PROJECT_ID)

# Load data from Google Earth Engine

In [ ]:
# Load and filter zip code feature collection
zips = ee.FeatureCollection('TIGER/2010/ZCTA5').filter(ee.Filter.lt("ALAND10", 10130491060))
print('num zips:', zips.size().getInfo())

In [ ]:
# Filter for contiguous US
contiguous_us = ee.Geometry.Rectangle([-127.18, 19.39, -62.75, 51.29])
zips = zips.filterBounds(contiguous_us)
print('contiguous US zips:', zips.size().getInfo())

In [ ]:
nighttime_lights = ee.ImageCollection('NOAA/VIIRS/DNB/ANNUAL_V22').select('median').median()  # 500m resolution

In [ ]:
def summarize_region(feature):
  feature = feature.simplify(50)

  def process_geometries(geometry):
    geometry = ee.Geometry(geometry)
    return ee.Algorithms.If(
        ee.String(geometry.type()).compareTo('Polygon'), None, geometry
    )

  if feature.geometry().type() != 'Polygon':
    filtered_geoms = (
        feature.geometry()
        .geometries()
        .map(process_geometries)
        .filter(ee.Filter.notNull(['item']))
    )
    feature = feature.setGeometry(ee.Geometry.MultiPolygon(filtered_geoms))

  # Clip datasets to feature
  lights = nighttime_lights.clip(feature.geometry()).reduceRegion(
      reducer=ee.Reducer.mean(), scale=500, maxPixels=1e9
  )

  # Set the computed values to the feature
  feature = feature.set({
      'night_lights': lights.get('median'),
  })
  return feature


# Polling function to check task status
def poll_task(task, task_name, interval=120):
  print(f'Polling {task_name} task...')
  while task.active():
    print(f"{task_name} task status: {task.status()['state']}")
    time.sleep(interval)
  final_status = task.status()['state']
  print(f'{task_name} task completed with status: {final_status}')

In [ ]:
data = zips.map(summarize_region)

In [ ]:
# Export results to drive
export_drive_task = ee.batch.Export.table.toDrive(
    collection=data,
    description='zipcode_environmental_simplified',
    folder='content',
    fileFormat='GeoJSON'
)
export_drive_task.start()

In [ ]:
import time
poll_task(export_drive_task, "Export to Drive")

# Using PDFM Embeddings and Nightime lights in a prediction task

#### The following cells in this notebook will access the PD-Foundations embeddings directly from a BigQuery table.

⚠️ Important: To run these cells successfully, you must first obtain access to the embeddings dataset via a BigQuery Listing. Please use [this form](https://forms.gle/ysdp5uUoPrMrhjZQA) to apply for research access or to join the Early Access Program waitlist.

Once your access is approved, ensure you are authenticated in this Colab with a Google account that has permission to query the BigQuery table..

In [ ]:
import pandas as pd

# @markdown Specify the BigQuery table ID for the embeddings.
# Example: 'your-gcp-project.your_dataset.your_table'
bigquery_table_id = '{PROJECT_ID}.pdfm_embeddings.us_embeddings_postal_code_v0' # @param {type:"string"}

# Construct the SQL query.
# This query selects all columns from the specified BigQuery table.
# Ensure your BigQuery table has a column named 'place' to be used as the index.
query = f"SELECT * FROM `{bigquery_table_id}`"

# Load data from BigQuery into a pandas DataFrame.
# pd.read_gbq handles authentication using your Colab credentials.
zipcode_embeddings = pd.read_gbq(query, project_id='pdfm-ttp-504664', dialect='standard').set_index('place_name')

In [ ]:
zipcode_embeddings.index = zipcode_embeddings.index.map(lambda x: f'zip/{x}')

In [ ]:
zipcode_embeddings.head(2)

In [ ]:
embedding_features = [f'feature{x}' for x in range(330)]

In [ ]:
#@title: Mount the drive folder which has the downloaded data from earth engine
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

## Load the night time lights data

In [ ]:
import geopandas as gpd

nighttime_lights_gdf = gpd.read_file('/content/drive/MyDrive/content/zipcode_environmental_simplified.geojson')
nighttime_lights_gdf['ZCTA5CE10'] =  nighttime_lights_gdf['ZCTA5CE10'].apply(lambda x: f'zip/{x}')
nighttime_lights_gdf.set_index('ZCTA5CE10', inplace=True)
nighttime_lights_gdf.index.name = 'place_name'

In [ ]:
nighttime_lights_gdf.head(2)


In [ ]:
nighttime_lights_df = pd.DataFrame({'place_name': nighttime_lights_gdf.index, 'night_lights': nighttime_lights_gdf['night_lights']}).reset_index(drop=True)
nighttime_lights_df.set_index('place_name', inplace=True)

## Visualization

In [ ]:
def get_locale(df, index, states=None, counties=None):
  df = df[df.index.isin(index)]
  if not states and not counties:
    return df
  filter = df.state.isin(states)
  if counties:
    filter &= df.county.isin(counties)
  return df[filter]

In [ ]:
df = zipcode_embeddings.join(nighttime_lights_gdf)
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [ ]:
feature = 'night_lights'
state = 'New York'

In [ ]:
df = zipcode_embeddings.join(nighttime_lights_gdf)
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf['state'] = gdf['location_metadata'].apply(lambda x: x.get('administrative_area_level1'))
gdf['county'] = gdf['location_metadata'].apply(lambda x: x.get('administrative_area_level2'))

In [ ]:
gdf.head()

In [ ]:
locale_gdf = get_locale(gdf, zipcode_embeddings.index, states=[state])

# Filter out rows with None or empty geometries to prevent errors in total_bounds calculation
locale_gdf = locale_gdf[~locale_gdf.geometry.is_empty & ~locale_gdf.geometry.isna()]

if not locale_gdf.empty:
  ax = locale_gdf.plot(feature, legend=True)
  _ = ax.set_title(feature + ' in zipcodes of ' + state)
else:
  print(f"No valid data found for state: {state}. Check the 'state' column in gdf and the content of location_metadata for inconsistencies.")

# Modelling

In [ ]:
from sklearn import metrics as skmetrics
import math
import matplotlib.pyplot as plt

def evaluate(df: pd.DataFrame) -> dict[float]:
  """Evaluates the model performance on the given dataframe.

  Args:
    df: A pandas DataFrame with columns 'y' and 'y_pred'.
  Returns:
    A dictionary of performance metrics.
  """

  df = df.dropna(subset='y')
  df = df[df.y != 0]
  r2 = skmetrics.r2_score(df.y, df.y_pred)
  r = df['y'].corr(df['y_pred'])
  rmse = math.sqrt(skmetrics.mean_squared_error(df.y, df.y_pred))
  mae = skmetrics.mean_absolute_error(df.y, df.y_pred)
  mape = skmetrics.mean_absolute_percentage_error(df.y, df.y_pred)
  return {'r2': r2, 'rmse': rmse, 'mae': mae, 'mape': mape, 'r': r}



def plot_actual_vs_predicted(df: pd.DataFrame):
    """Plots actual vs. predicted values to visualize model performance.

    Args:
      df: A pandas DataFrame with columns 'y' and 'y_pred'.
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(df['y'], df['y_pred'], alpha=0.5)
    plt.plot([df['y'].min(), df['y'].max()], [df['y'].min(), df['y'].max()], 'r--', linewidth=2)  # Reference line
    plt.xlabel('Actual Values (y)')
    plt.ylabel('Predicted Values (y_pred)')
    plt.title('Actual vs. Predicted Values')
    plt.grid(True)
    plt.show()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

data = zipcode_embeddings.merge(nighttime_lights_df, left_index=True, right_index=True)
label = 'night_lights'

X = data['features']
y = data[label]

# Convert the Series of lists into a DataFrame where each list element is a column
X = pd.DataFrame(X.tolist(), index=X.index)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train a simple linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

evaluation_df = pd.DataFrame({'y': y_test, 'y_pred': y_pred})
# Evaluate the model
metrics = evaluate(evaluation_df)
print(metrics)
plot_actual_vs_predicted(evaluation_df)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

k = 5
model = KNeighborsRegressor(n_neighbors=k)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

evaluation_df = pd.DataFrame({'y': y_test, 'y_pred': y_pred})
# Evaluate the model
metrics = evaluate(evaluation_df)
print(metrics)
plot_actual_vs_predicted(evaluation_df)